In [24]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import os
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)
dataroot = "../data/"
cacheRoot = "../cache/"
subRoot = "../submission/"

In [19]:
# import os

# def list_files(startpath):
#     for root, dirs, files in os.walk(startpath):
#         level = root.replace(startpath, '').count(os.sep)
#         indent = ' ' * 4 * (level)
#         print('{}{}/'.format(indent, os.path.basename(root)))
#         subindent = ' ' * 4 * (level + 1)
#         for f in files:
#             print('{}{}'.format(subindent, f))
#list_files("../")
def get_embeding(fname, embname):
    f = open(fname)
    embeding_lines = f.readlines()
    f.close()
    mapfunc = lambda x: list( map( float, x ) )
    embeding_lines = [li.replace("\n","").split(" ") for li in embeding_lines[1:]]
    embeding_lines = [ [ int(line[0]) ] +  mapfunc( line[1:]  )   for line in   embeding_lines ]
    cols = ["UID"] + [ embname  + str(i) for i in range( len(embeding_lines[0]) -1 )]
    embeding_df = pd.DataFrame(embeding_lines, columns=cols )
    del embeding_lines
    return embeding_df



/
    0-运行说明.txt
    1-模型说明.txt
cache/
    code2_weighted_edglist_DeepWalk.txt
    code2_weighted_edglist_filytypeTxt.edgelist
    code2_weighted_edglist_filytypeTxt.txt
    device1_weighted_edglist_DeepWalk.txt
    device1_weighted_edglist_filytypeTxt.edgelist
    device1_weighted_edglist_filytypeTxt.txt
    device_code1_weighted_edglist_DeepWalk.txt
    device_code1_weighted_edglist_filytypeTxt.edgelist
    device_code1_weighted_edglist_filytypeTxt.txt
    device_code2_weighted_edglist_DeepWalk.txt
    device_code2_weighted_edglist_filytypeTxt.edgelist
    device_code2_weighted_edglist_filytypeTxt.txt
    device_code3_weighted_edglist_DeepWalk.txt
    device_code3_weighted_edglist_filytypeTxt.edgelist
    device_code3_weighted_edglist_filytypeTxt.txt
    geo_code_weighted_edglist_DeepWalk.txt
    geo_code_weighted_edglist_filytypeTxt.edgelist
    geo_code_weighted_edglist_filytypeTxt.txt
    grouping_features.csv
    ip1_weighted_edglist_DeepWalk.txt
    ip1_weighted_edglist_filytype

In [71]:
data =  pd.read_csv(cacheRoot + "grouping_features.csv")

In [103]:
mac1_emb = get_embeding(cacheRoot + "mac1_.emb", "mac1_emb_")
merchant_dbk = get_embeding(cacheRoot + "merchant_weighted_edglist_DeepWalk.embeddings", "merchant_deepwalk_")
mac1_emb = mac1_emb[ mac1_emb.UID.map(lambda x: x<= 131587) ] 
merchant_dbk = merchant_dbk[ merchant_dbk.UID.map(lambda x: x<= 131587) ] 


merchant_emb = get_embeding(cacheRoot + "merchant_.emb", "merchant_emb_")
merchant_emb = merchant_emb[ merchant_emb.UID.map(lambda x: x<= 131587) ] 
mac_merch = pd.DataFrame()
mac_merch["UID"] = merchant_dbk.UID.tolist() +  mac1_emb.UID.tolist()
mac_merch = mac_merch.merge( mac1_emb, on = ["UID"],how = "left"  )
mac_merch = mac_merch.merge( merchant_dbk, on = ["UID"],how = "left"  )
mac_merch = mac_merch.fillna( 0.0 )

In [53]:
# len(mac_merch.columns)

In [104]:
# for embedding mac1 and merchants pca
from sklearn.decomposition import PCA
dim = 42
pca = PCA(n_components=dim)
pca_res = pca.fit_transform(mac_merch.drop("UID", axis = 1)  )
me_pca = pd.DataFrame(pca_res, columns=["pca_mac1_merchrant_%d" % i for i in range(dim)])
me_pca["UID"] = mac_merch.UID.values

In [57]:
# datapca.head()

In [105]:
datapca = pd.merge( data,me_pca,on = ["UID"],how = "left"  )
data = pd.merge( data, mac1_emb, on =["UID"], how="left" )
data = pd.merge( data, merchant_dbk, on =["UID"], how="left" )

In [64]:
merchant_dbk.head()

,UID,merchant_emb_0,merchant_emb_1,merchant_emb_2,merchant_emb_3,merchant_emb_4,merchant_emb_5,merchant_emb_6,merchant_emb_7,merchant_emb_8,...,merchant_emb_26,merchant_emb_27,merchant_emb_28,merchant_emb_29,merchant_emb_30,merchant_emb_31,merchant_emb_32,merchant_emb_33,merchant_emb_34,merchant_emb_35
0,86621,0.003960,0.006373,-0.009713,0.009350,-0.005658,0.010234,0.011375,-0.001988,0.009586,...,0.001839,-0.010012,0.013202,0.012239,0.007381,-0.001205,-0.009609,-0.000264,-0.006994,0.009123
1,100269,0.001309,0.005653,-0.008265,0.010637,0.012041,0.013026,-0.001788,-0.012769,-0.011668,...,-0.006183,-0.000730,0.013126,-0.013579,-0.004607,-0.003498,0.011188,0.000615,0.001480,-0.001900
2,60349,0.008413,-0.006961,-0.003838,0.010695,-0.003668,0.004025,0.001820,0.008155,0.011400,...,0.008021,0.011304,0.013346,0.006140,0.002814,-0.002084,-0.012451,-0.009636,-0.005739,0.007469
5,81571,-0.006576,-0.001043,0.003041,-0.008475,-0.003729,0.012110,0.006799,0.010017,0.008056,...,-0.001775,-0.005877,-0.010731,-0.005589,0.010125,0.000915,-0.006479,-0.001277,0.009272,-0.001842
6,81942,0.002408,-0.005394,-0.009702,0.004217,0.002290,0.009670,-0.009860,0.001250,0.006747,...,-0.010634,0.009837,0.010011,-0.003991,-0.011226,-0.001347,0.008973,0.002624,0.004127,-0.002723


In [73]:
import os
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from sklearn.svm import SVC
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['Tag']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 
def cv(df, num_folds, param, stratified=True, debug=False):
    train_df =  df[df.Tag != 0.5]
    test_df =   df[df.Tag == 0.5]
    
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['Tag'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['Tag'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['Tag'].iloc[valid_idx]
        fold_preds = test_df[["UID"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(param,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['Tag'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['Tag'], oof_preds)
    all_test_preds = pd.concat(all_test_preds, axis=0)
    sub = pd.DataFrame()
    sub['UID'] = all_test_preds.UID.unique()
    sub.set_index("UID", inplace=True)
    sub["Tag"] = all_test_preds.groupby("UID").Tag.mean()
    print('Full AUC score %.6f' % full_auc)
    
    return [full_auc,sub]

In [107]:
class runmodel(object):
    def __init__(self,data):
        self.data = data
    def getinfo(self):
        print("use variable = runlgb.get_result() get reusult ")
    def get_result(self):
        params_list =[
         {'boosting_type': 'goss', 'colsample_bytree': 0.6555004575000242, 'learning_rate': 0.016380004820033073, 'max_bin': 1000, 'metric': 'auc', 'min_child_weight': 2.1115838168176433, 'num_leaves': 108, 'reg_alpha': 23.247001339889128, 'reg_lambda': 997.9576062039534, 'subsample': 1.0, 'verbose': 1, 'seed': 4419},
         {'boosting_type': 'goss', 'colsample_bytree': 0.7187703092392053, 'learning_rate': 0.01939219215282862, 'max_bin': 310, 'metric': 'auc', 'min_child_weight': 2.66983907940641, 'num_leaves': 89, 'reg_alpha': 18.48224434106526, 'reg_lambda': 470.54675380054465, 'subsample': 1.0, 'verbose': 1, 'seed': 57},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6274617979582582, 'learning_rate': 0.01680918441243103, 'max_bin': 780, 'metric': 'auc', 'min_child_weight': 0.8226071606806127, 'num_leaves': 73, 'reg_alpha': 14.466924422050258, 'reg_lambda': 658.5772060624658, 'subsample': 1.0, 'verbose': 1, 'seed': 1732},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6991838451153098, 'learning_rate': 0.01577419276366034, 'max_bin': 940, 'metric': 'auc', 'min_child_weight': 7.758954855388241, 'num_leaves': 187, 'reg_alpha': 43.60868666926589, 'reg_lambda': 667.6371302027073, 'subsample': 1.0, 'verbose': 1, 'seed': 139},
         {'boosting_type': 'goss', 'colsample_bytree': 0.6198406436401038, 'learning_rate': 0.016662891953242748, 'max_bin': 820, 'metric': 'auc', 'min_child_weight': 3.5602833459924015, 'num_leaves': 77, 'reg_alpha': 13.398041512170746, 'reg_lambda': 631.8105595021391, 'subsample': 1.0, 'verbose': 1, 'seed': 7609}
        ]
        result = []
        for i, params in enumerate(params_list):
            model_dir = "./model_output/random/lgb_693/%d/" % i
            result.append(cv(self.data , 5, params))
        return result


use variable = runlgb.get_result() get reusult 


In [108]:
runlgb = runmodel(data)
runlgb.getinfo()
sub_embeding = runlgb.get_result()

all_subs = [x[1] for x in sub_embeding]
all_sub3 = pd.concat( all_subs  )
all_sub3 = all_sub3.groupby("UID")["Tag"].agg({"Tag":"mean" }).reset_index()
all_sub3.Tag = all_sub3.Tag * 2.5
all_sub3.to_csv(subRoot + "sub2.csv", encoding = 'utf8', index = False)

(44898, 505) (11225, 505) (56043, 505)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.972327	valid's auc: 0.958134
[200]	train's auc: 0.983464	valid's auc: 0.969886
[300]	train's auc: 0.989222	valid's auc: 0.974807
[400]	train's auc: 0.992784	valid's auc: 0.978233
[500]	train's auc: 0.994968	valid's auc: 0.980708
[600]	train's auc: 0.996394	valid's auc: 0.982409
[700]	train's auc: 0.997366	valid's auc: 0.983828
[800]	train's auc: 0.99802	valid's auc: 0.984617
[900]	train's auc: 0.998447	valid's auc: 0.985465
[1000]	train's auc: 0.99876	valid's auc: 0.985949
[1100]	train's auc: 0.998999	valid's auc: 0.986314
[1200]	train's auc: 0.999166	valid's auc: 0.986958
[1300]	train's auc: 0.99929	valid's auc: 0.987223
[1400]	train's auc: 0.999389	valid's auc: 0.987607
[1500]	train's auc: 0.999464	valid's auc: 0.987787
[1600]	train's auc: 0.999524	valid's auc: 0.987913
[1700]	train's auc: 0.999565	valid's auc: 0.988133
[1800]	train's auc: 0.9996	valid's auc: 0.9

D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.988865
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.971339	valid's auc: 0.959317
[200]	train's auc: 0.983008	valid's auc: 0.969787
[300]	train's auc: 0.989172	valid's auc: 0.974557
[400]	train's auc: 0.99265	valid's auc: 0.977943
[500]	train's auc: 0.99487	valid's auc: 0.980249
[600]	train's auc: 0.996312	valid's auc: 0.981773
[700]	train's auc: 0.997337	valid's auc: 0.983113
[800]	train's auc: 0.99799	valid's auc: 0.984095
[900]	train's auc: 0.998439	valid's auc: 0.984889
[1000]	train's auc: 0.998772	valid's auc: 0.985657
[1100]	train's auc: 0.999006	valid's auc: 0.986087
[1200]	train's auc: 0.999186	valid's auc: 0.986341
[1300]	train's auc: 0.999316	valid's auc: 0.986802
[1400]	train's auc: 0.999422	valid's auc: 0.987018
[1500]	train's auc: 0.999497	valid's auc: 0.987245
[1600]	train's auc: 0.999558	valid's auc: 0.987556
[1700]	train's auc: 0.999612	valid's auc: 0.98771
[1800]	train's auc: 0.999652	valid's auc: 0.987813
[1900]	tr

D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  2 AUC : 0.988462
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.930622	valid's auc: 0.922746
Early stopping, best iteration is:
[61]	train's auc: 0.964707	valid's auc: 0.952801


...

Fold  3 AUC : 0.987384
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.934406	valid's auc: 0.927676
Early stopping, best iteration is:
[60]	train's auc: 0.966591	valid's auc: 0.959083


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  4 AUC : 0.959083
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.976768	valid's auc: 0.962903
[200]	train's auc: 0.987589	valid's auc: 0.973429
[300]	train's auc: 0.992646	valid's auc: 0.978211
[400]	train's auc: 0.99546	valid's auc: 0.981027
[500]	train's auc: 0.997076	valid's auc: 0.982816
[600]	train's auc: 0.998051	valid's auc: 0.984575
[700]	train's auc: 0.998649	valid's auc: 0.985688
[800]	train's auc: 0.999015	valid's auc: 0.986637
[900]	train's auc: 0.999257	valid's auc: 0.987474
[1000]	train's auc: 0.99942	valid's auc: 0.987757
[1100]	train's auc: 0.999536	valid's auc: 0.988392
[1200]	train's auc: 0.999618	valid's auc: 0.988823
[1300]	train's auc: 0.999683	valid's auc: 0.989005
Early stopping, best iteration is:
[1277]	train's auc: 0.999671	valid's auc: 0.989054


D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  5 AUC : 0.989054
Full AUC score 0.984917


In [ ]:
runlgbpca = runmodel(datapca)
runlgbpca.getinfo()
sub_embedingpca = runlgbpca.get_result()

all_subspca = [x[1] for x in sub_embedingpca]
all_subspca = pd.concat( all_subs  )
all_subspca = all_subspca.groupby("UID")["Tag"].agg({"Tag":"mean" }).reset_index()
all_subspca.to_csv(subRoot + "sub3.csv", encoding = 'utf8', index = False)